In [ ]:
from cftime import DatetimeNoLeap
from datetime import timedelta
from os import chdir, system
from mpl_toolkits.basemap import interp
from numpy import datetime64, empty, full, repeat, stack
from scipy.interpolate import griddata
from xarray import DataArray, open_dataset

**Data**

In [ ]:
nc_path = '/home/zhangc/scenariomip_cmip6/nc_data/'
im_path = '/home/zhangc/scenariomip_cmip6/im_data/'
os_path = '/home/zhangc/repositories/nc2im_cesm2/'

In [ ]:
nc_con = ['fracdata_0.9x1.25_gx1v6_c090317.nc',
          'USGS-gtopo30_0.9x1.25_remap_c051027.nc']

In [ ]:
nc_6hr = ['ta_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'hus_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'ua_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'va_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc',
          'ps_6hrLev_CESM2_ssp245_r2i1p1f1_gn_201501010000-202412311800.nc']

In [ ]:
nc_day = ['tos_Oday_CESM2_ssp245_r2i1p1f1_gn_20150102-20650101.nc',
          'siconc_SIday_CESM2_ssp245_r2i1p1f1_gn_20150102-20650101.nc']

In [ ]:
nc_mon = ['ts_Amon_CESM2_ssp245_r2i1p1f1_gn_201501-206412.nc']

In [ ]:
nc_era = ['soil_mon_ERA5_2015.nc']

In [ ]:
chdir(nc_path)

**Time**

In [ ]:
tnum = 2015, 1, 1
t6hr = DatetimeNoLeap(tnum[0], tnum[1], tnum[2])
tday = t6hr + timedelta(days=1)
tmon = DatetimeNoLeap(tnum[0], tnum[1], 15, 12)
tera = datetime64('2015-'+str(tnum[1]).zfill(2)+'-01')
file_time = str(t6hr).replace(' ', '_')

**Constant**

In [ ]:
ds = open_dataset(nc_con[0])
da = ds.LANDMASK*1.0
file_name = 'ls_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

In [ ]:
ds = open_dataset(nc_con[1])
da = ds.PHIS/9.81
file_name = 'sh_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**6 Hourly**

In [ ]:
das = []

In [ ]:
for nc in nc_6hr:
    ds = open_dataset(nc).sel(time=t6hr)
    vi = ds.variable_id
    da = ds[vi]
    das.append(da)
    file_name = vi + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

**3D Geopotential Heights** Virtual Temperature

In [ ]:
t = das[0].sortby('lev').values
q = das[1].sortby('lev').values
tv = t*(1.+q*0.61)

In [ ]:
lev = -das[0].sortby('lev').lev.values
lat = das[0].lat.values
lon = das[0].lon.values

In [ ]:
da = DataArray(name='tv', data=tv, coords=[lev, lat, lon], dims=['lev', 'lat', 'lon']).transpose('lat', 'lon', 'lev')
file_name = 'tv_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**3D Geopotential Heights** Pressure

In [ ]:
shape = das[0].shape
p3 = empty(shape)
for i in range(len(lev)):
    p3[i] = full(shape[1:3], lev[i])

In [ ]:
da = DataArray(name='p3', data=p3, coords=[lev, lat, lon], dims=['lev', 'lat', 'lon']).transpose('lat', 'lon', 'lev')
file_name = 'p3_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**Surface Variables**

In [ ]:
vis = ['t2', 'q2', 'u2', 'v2']

In [ ]:
for i in range(4):
    da = das[i].isel(lev=-1)
    file_name = vis[i] + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

In [ ]:
for nc in nc_mon:
    ds = open_dataset(nc).sel(time=tmon)
    vi = ds.variable_id
    da = ds[vi]
    file_name = vi + '_' + file_time + '.nc'
    da.to_netcdf(im_path+file_name)

In [ ]:
da = das[0].isel(lev=-1)
lev = da.lev.values*-100
lon = da.lon.values
lat = da.lat.values
values = full(da.shape, lev)
da = DataArray(name='p2', data=values, coords=[lat, lon], dims=['lat', 'lon'])
file_name = 'p2_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**Sea Surface Temperature**

In [ ]:
ds = open_dataset(nc_day[0])
da = ds.tos.sel(time=tday)
x = da.lon.values.flatten()
y = da.lat.values.flatten()
xy = stack((x, y), axis=-1)
v = da.values.flatten()

In [ ]:
ds = open_dataset(nc_6hr[0])
lon1d = ds.lon.values
lat1d = ds.lat.values
nx = len(lon1d)
ny = len(lat1d)
lon2d = repeat(lon1d, ny).reshape(nx, ny).transpose()
lat2d = repeat(lat1d, nx).reshape(ny, nx)

In [ ]:
vi = griddata(xy, v, (lon2d, lat2d), method='linear')
da = DataArray(data=vi, coords=[lat1d, lon1d], dims=['lat', 'lon'])
file_name = 'sst_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**Sea Ice Concentration**

In [ ]:
ds = open_dataset(nc_day[1])
da = ds.siconc.sel(time=tday)
v = da.values.flatten()

In [ ]:
vi = griddata(xy, v, (lon2d, lat2d), method='linear')
da = DataArray(data=vi, coords=[lat1d, lon1d], dims=['lat', 'lon'])
file_name = 'sic_' + file_time + '.nc'
da.to_netcdf(im_path+file_name)

**Soil Moisture**

In [ ]:
ds = open_dataset(nc_era[0]).sel(time=tera)

In [ ]:
x1d = ds.longitude.values
y1d = ds.latitude.values
nx = len(x1d)
ny = len(y1d)

In [ ]:
x2d = repeat(x1d, ny).reshape(nx, ny).transpose()
y2d = repeat(y1d, nx).reshape(ny, nx)

In [ ]:
x2df = x2d.flatten()
y2df = y2d.flatten()
xy = stack((x2df, y2df), axis=-1)

In [ ]:
da = ds[vis[0]]

In [ ]:
di = da.interp(longitude=lon, latitude=lat)

In [ ]:
da

In [ ]:
lon

In [ ]:
di.plot()

**Soil Temperature**

In [ ]:
v = da.values.flatten()
vi = griddata(xy, v, (lon2d, lat2d), method='linear')

In [ ]:
vis = ['swvl1', 'swvl2', 'swvl3', 'swvl4']
for vi in vis:
    da = ds[vi]
    v = da.values.flatten()
    vi = griddata(xy, v, (lon2d, lat2d), method='linear')

In [ ]:
#chdir(os_path)

In [ ]:
#command = 'ncl convert_nc_to_im.ncl ' + "'file_time=" + '"' + file_time + '"' + "'"

In [ ]:
#system(command)